## Análisis de Datos Faltantes  

In [1]:
# Librerias
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
#import session_info
import upsetplot

In [2]:
# linea para correr el notebook utils
%run utils.ipynb

### Datos. Descarga de archivos desde una URL

#### 1. Creamos unidades de informacion de los conjuntos de datos

In [16]:
# Guardar la url en una variable
datos_url = 'https://github.com/njtierney/naniar/raw/master/data/'

# Almacenar los nombres de los archivos a descargar en una lista
datasets_names = ('oceanbuoys', 'pedestrian', 'riskfactors')

# Almacenar el tipo de extension en una variable
extension = '.rda'

#### 2. Descargar y cargar los conjuntos de datos 

In [17]:
datasets_dfs = {} # diccionario que almacena los archivos descargados

for dataset_name in datasets_names:

    dataset_file = f'{ dataset_name }{ extension }' # almacena el nombre de cada archivo con su respectiva extension
    dataset_output_file = f'./data/{ dataset_file }' # ruta y nombre donde se guarda los arhcivos
    dataset_url = f'{ datos_url }{ dataset_file }' # url de cada archivo a descargar

    !wget -q -O { dataset_output_file } { dataset_url } # linea que descarga los archivos

    # almacenar en el diccionario los archivos descargados
    datasets_dfs[f'{ dataset_name }_df'] = pyreadr.read_r(dataset_output_file).get(dataset_name)

# accede a las llaves del diccionario para verificar que los archivos se hayan cargado
datasets_dfs.keys()

dict_keys(['oceanbuoys_df', 'pedestrian_df', 'riskfactors_df'])

#### 3. Incluir conjunto de datos en nuestro ambiente local

In [20]:
locals().update(**datasets_dfs)
del datasets_dfs # elimina el diccionario dejando los archivos df

In [22]:
riskfactors_df.shape, oceanbuoys_df.shape, pedestrian_df.shape

((245, 34), (736, 8), (37700, 9))